<a href="https://colab.research.google.com/github/vvmnnnkv/private-ai/blob/master/Section%201%20-%20Differential%20Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Generate Parallel Databases


In [0]:
import Foundation
import TensorFlow

In [0]:
typealias Database = Tensor<Float>
typealias Query = (_ db: Database) -> Float

/// Return vector of 0 and 1 from uniform distibution
func createRandomBinaryVector(of shape: TensorShape) -> Database {
  let dist = UniformIntegerDistribution<UInt8>(lowerBound: 0, upperBound: 1)
  var scalars: [Float] = []
  for _ in 0 ..< shape.contiguousSize {
     scalars.append(Float(dist.next(using: &PhiloxRandomNumberGenerator.global)))
  }
  return Tensor(shape: shape, scalars: scalars)  
}


In [0]:
/// Create parallel DB from original DB
func createParallelDb<T>(from tensor: Tensor<T>, dropping index: Int) -> Tensor<T> {
  var top = tensor.slice(lowerBounds: [0], upperBounds: [index])
  var bottom = tensor.slice(lowerBounds: [index + 1], upperBounds: [tensor.shape[0]])
  return top.concatenated(with: bottom)
}

In [0]:
/// Create all possible parallel DBs
func createParallelDbs<T>(from tensor: Tensor<T>) -> Array<Tensor<T>> {
  var result = Array<Tensor<T>>()
  for i in 0 ..< tensor.shape[0] {
    let pdb = createParallelDb(from: tensor, dropping: i)
    result.append(pdb)
  }
  return result
}

In [0]:
/// Create DB and parallal DBs for given number of entries
func createDbAndParallels(entries: Int) -> (db: Database, pdbs: Array<Database>) {
  let db = createRandomBinaryVector(of: [entries])
  let pdbs = createParallelDbs(from: db)
  return (db: db, pdbs: pdbs)
}

In [0]:
var (db, dbs) = createDbAndParallels(entries: 1000)

In [7]:
print(db.shape)
print(dbs.count)
print(dbs[0].shape)

[1000]
1000
[999]


# Project: Evaluating the Privacy of a Function

In [0]:
/// Calculate empirical sensitivity of a query
func sensitivity(of query: Query, with entries: Int) -> Float {
  var (db, pdbs) = createDbAndParallels(entries: entries)
  let dbQuery = query(db)
  var maxDifference:Float = 0
  for pdb in pdbs {
    let pdbQuery = query(pdb)
    let difference = abs(dbQuery - pdbQuery)
    if (maxDifference < difference) {
      maxDifference = difference
    }
  }
  return maxDifference
}

In [0]:
/// Sum query
let sumQuery:Query = { db in db.sum().scalar! }

In [10]:
sensitivity(of: sumQuery, with: 5000)

1.0


In [0]:
/// Mean query
let meanQuery:Query = { db in db.mean().scalar! }

In [15]:
sensitivity(of: meanQuery, with: 5000)

0.00010162592


In [20]:
sensitivity(of: meanQuery, with: 100)

0.00535354


# Project: Calculate L1 Sensitivity For Threshold

In [0]:
/// General threshold of sum func
func threshold(_ db: Database, _ threshold: Float) -> Int {
  let sum = db.sum()
  return sum > threshold ? 1 : 0
}

/// Threshold is set to 5
let threshold5Query: Query = { db in Float(threshold(db, 5)) }

In [18]:
for _ in 0..<10 {
  print(sensitivity(of: threshold5Query, with: 10))
}

0.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
